# Fitting AUSF

The model requires the fitting of pig population dynamics and ASF disease dynamics. We first fit the population to ensure that the population returns to the carrying capacity

### Load Modules

In [ ]:
using GpABC, Distances, Distributions, Plots, StatsBase, Printf
using DiffEqCallbacks, DifferentialEquations, LinearAlgebra
using DelimitedFiles, DataFrames, LinearAlgebra, SparseArrays


In [ ]:
include("/home/callum/ASF/Modules/ASF_input.jl");
include("/home/callum/ASF/Modules/ASF_model.jl");
include("/home/callum/ASF/Modules/ASF_analysis.jl")

In [ ]:
input_path = "/home/callum/ASF/Inputs/"; #path to model data

### Model

In [ ]:
input = ASF_Inputs.Model_Data(input_path, true); #loading the model input

In [ ]:
const nt = input.Parameters.Populations.cum_sum[end] #total number of groups and farms
const nc = 5 #number of classes
const eqs = 11 #number of processes

const dc = sparse(zeros(nt*nc,nt*eqs))

#const _dc = zeros(nt*5,nt*10)

dc[0*nc*nt+1:nc*nt*eqs+nc:end] .= 1
dc[1*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[2*nc*nt+1:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+1:nc*nt*eqs+nc:end] .= 1

dc[2*nc*nt+2:nc*nt*eqs+nc:end] .= 1
dc[3*nc*nt+2:nc*nt*eqs+nc:end] .= -1
dc[4*nc*nt+2:nc*nt*eqs+nc:end] .= -1

dc[4*nc*nt+3:nc*nt*eqs+nc:end] .= 1
dc[5*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[6*nc*nt+3:nc*nt*eqs+nc:end] .= -1
dc[7*nc*nt+3:nc*nt*eqs+nc:end] .= -1

dc[7*nc*nt+4:nc*nt*eqs+nc:end] .= 1
dc[8*nc*nt+4:nc*nt*eqs+nc:end] .= -1
dc[10*nc*nt+4:nc*nt*eqs+nc:end] .= -1


dc[5*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[6*nc*nt+5:nc*nt*eqs+nc:end] .= 1
dc[9*nc*nt+5:nc*nt*eqs+nc:end] .= -1;

In [ ]:
function regular_c(du,u,p,t,counts,mark)  
    mul!(du,dc,counts)
    nothing
end

### Fitting Population

In [ ]:
AUSF_pop = function(params,Tspan, U0)
    #function that runs model for fitting

    rj = RegularJump(ASF_Model.asf_model_pop, regular_c, eqs*nt)

    prob = DiscreteProblem(U0,Tspan,params)

    jump_prob = JumpProblem(prob,Direct(),rj)

    ensemble_prob_m = EnsembleProblem(jump_prob)

    outputs = solve(ensemble_prob_m, SimpleTauLeaping(), trajectories= n_runs,EnsembleThreads(),dt=1);

    res = eval_out_pop(outputs,U0)

    return res
end
    

function simulator_function_pop(var_params)

   
    input = ASF_Inputs.Model_Data(input_path);

    U0 = copy(input.U0);
    U0[1:5:end] += (U0[2:5:end] +U0[3:5:end]) #setting init pop to zero
    U0[2:5:end] .= 0
    U0[3:5:end] .= 0;
    
    params = ASF_Model.convert(input.Parameters)
    
    params[7] = var_params[1] #corpse decay rates
    
    Time = (0.0,n_years*365)
    #now we run simulation!
    AUSF_pop(params, Time, U0)
end

function eval_out_pop(output, U0)
    output_final = Matrix{Float64}(undef,(n_years*365+1,1))
    dist = zeros(Float64,n_runs)
    K = sum(U0)
   dummy = zeros(n_years*365+1)
    
    for i = 1:n_runs
        sol = output[i]
        data = reduce(vcat,transpose.(sol.u))
        data[data .< 0 ] .= 0

        s_d = data[:,1:5:end]
        
        pop_sum = sum(s_d,dims=2)[:,1]
        
        pop_crop = pop_sum/K
 
        dummy += pop_crop/n_runs
        
    
    end
    output_final[:] = dummy
    dist = euclidean(obs[:],dummy)
    if euclidean(obs[:],dummy) < threshold
        println(dist)
    end
    return output_final
    
end

function observed_params()
   
    obs =  Matrix{Float64}(undef,(n_years*365+1,1))
    
    obs .= 1
    
    return obs
end

In [ ]:
#Some key parameters for G

input_path = "/home/callum/ASF/Inputs/"; #path to model data
n_years = 5 #how many years each sim will run for
n_runs = 3 #how many runs for each param
obs = observed_params() #our observed (just array of ones, we normalise output to be relative to this)
priors = [Uniform(5, 10)];

In [ ]:
#Our params for fitting
n_particles = 10 #number of acceptances we want
devi = 0.99 #how close (from 0-1 with 1 being identical) do we want output to be to observed, used for threshold
threshold = sqrt((n_years*365+1)*((1-devi)^2));

In [ ]:
sim_result = SimulatedABCRejection(obs, simulator_function_pop, priors, threshold, n_particles; max_iter=1000, write_progress=false)#,

### Fitting ASF

Here we are fitting the diease dynamics to previously observed effects from outbreaks in the Baltic States

In [ ]:
function AUSF_d(dis_params,Tspan,U0)
     rj = RegularJump(ASF_Model.asf_model_one, regular_c, eqs*nt)
    prob = DiscreteProblem(U0,Tspan,dis_params)
    jump_prob = JumpProblem(prob,Direct(),rj)
    ensemble_prob = EnsembleProblem(jump_prob)
    model_outs = solve(ensemble_prob, SimpleTauLeaping(), trajectories= n_runs,EnsembleThreads(),dt=1)
    res = eval_out(model_outs)
end
    
function simulator_function_d(var_params)
    Tspan = (0.0,365*n_years)
    
    input = ASF_Inputs.Model_Data(input_path);
    U0 = input.U0
    params = ASF_Model.convert(input.Parameters)
    
    #beta
    params[1] .= var_params[1] #intra
    params[2][params[2] .!= 0 ] .= var_params[2]/n_con #inter
    
    #corpse 
    params[10] = var_params[3] #corpse infection modifier

    AUSF_d(params, Tspan, U0)
end

function eval_out(data) 

    output_final = Matrix{Float64}(undef,(3,1))
    s = 0
    ep = 0
    mt = 0
    pd = 0
    
    for i in 1:n_runs
        d, da,f,p = run_analysis(data[i])

        if d[end] > 0

            ep += 100*mean(da[3*365:end])/mean(p[3*365:end])
            pd += 100*(1-mean(p[3*365:end])/p[1])
            mt += maximum(findall(==(maximum(d)), d))[1]


            s += 1
        end

    end
    
    
    if s >= 2
        output_final[1] = ep/s*(1/2)
        output_final[2] = pd/s*(1/10)
        output_final[3] = mt/s*(2/365)
        
    else
        output_final[1] = 0
        output_final[2] = 0
        output_final[3] = 0
    end
    
    d= euclidean(output_final,obs)
    if d < threshold
        println(d)
    end

    return output_final
    
end

function observed_params()
   
    obs =  Matrix{Float64}(undef,(3,1))
    
    obs[1] = 1 #2 #endemic prev
    obs[2] = 8#80 #population delcine %
    obs[3] = 2#180 #time (days) from init to max
    
    return obs
end

In [ ]:
input_path = "/home/callum/ASF/Inputs/"; #path to model data
n_years = 5 #how many years each sim will run for
n_runs = 3 #how many runs for each param
n_con = 8  # average number of connections each group has
obs = observed_params() #our observed (just array of ones, we normalise output to be relative to this)
priors = [Uniform(0.15, 0.85), Uniform(0.015, 0.085), Uniform(0.15, 1)];

In [ ]:
n_particles = 100;
threshold_schedule = [1.0, 0.8, 0.7];

In [ ]:
sim_abcsmc_res = SimulatedABCSMC(obs,simulator_function_d,priors,threshold_schedule,n_particles; write_progress=false)